In [1]:
import pandas as pd
import numpy as np

In [2]:
final = pd.read_csv("alldata.csv")

In [5]:
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11189 entries, 0 to 11188
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   POI_name                11189 non-null  object 
 1   Average_Rating          11189 non-null  float64
 2   Destination_name        11189 non-null  object 
 3   POI_reviews_count       11189 non-null  int64  
 4   POI_category            11189 non-null  object 
 5   POI_description         6248 non-null   object 
 6   POI_rating              11189 non-null  float64
 7   POI_address             11189 non-null  object 
 8   POI_Suggested_Duration  2762 non-null   object 
 9   POI_open_close_hours    2977 non-null   object 
 10  POI_reviews             11189 non-null  object 
 11  Review_Sentiment        11189 non-null  object 
 12  Trip_type               11189 non-null  object 
dtypes: float64(2), int64(1), object(10)
memory usage: 1.1+ MB


In [6]:
final = final.drop( final[final['Review_Sentiment'] == "Negative"].index)
final = final.drop( final[final['Review_Sentiment'] == "Neutral"].index)

In [10]:
final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10095 entries, 0 to 11188
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   POI_name                10095 non-null  object 
 1   Average_Rating          10095 non-null  float64
 2   Destination_name        10095 non-null  object 
 3   POI_reviews_count       10095 non-null  int64  
 4   POI_category            10095 non-null  object 
 5   POI_description         5666 non-null   object 
 6   POI_rating              10095 non-null  float64
 7   POI_address             10095 non-null  object 
 8   POI_Suggested_Duration  2493 non-null   object 
 9   POI_open_close_hours    2727 non-null   object 
 10  POI_reviews             10095 non-null  object 
 11  Review_Sentiment        10095 non-null  object 
 12  Trip_type               10095 non-null  object 
dtypes: float64(2), int64(1), object(10)
memory usage: 1.1+ MB


In [42]:
final_norev = final.drop('POI_reviews', axis=1)

In [43]:
final_norev.shape

(10095, 12)

In [46]:
print(final.shape)
print(final["POI_name"].nunique())

(10095, 13)
479


In [47]:
final_norev.duplicated().sum()

9529

In [52]:
dd = final_norev.drop_duplicates()

In [54]:
dd["POI_name"].nunique()

479

In [60]:
dd = dd.drop_duplicates(subset='POI_name', keep="first")

In [61]:
dd.shape

(479, 12)

In [67]:
dd.to_csv (r'C:\Users\Anurag\Documents\TA_data\New data\All_Data\Preprocessed_data\POI_unique.csv', index = False, header=True)

#### Function to get recommended cities

In [13]:
POI_TripTypes_count = pd.read_csv("POI_TripTypes_count.csv")
def recc_city(trip_type_list):
    recc_city_df = (POI_TripTypes_count.loc[(POI_TripTypes_count["Trip_type"].isin(trip_type_list))]).groupby(["Destination_name"]).sum().nlargest(3,"Count")
    return list(recc_city_df.index)

In [27]:
nlp = ["Historic", "Relaxed"]
rec_cities = recc_city(nlp)
rec_cities

['Napa', 'Los Angeles', 'Santa Barbara']

In [68]:
POI_unique = pd.read_csv("POI_unique.csv")

## Extracting POI for recommended destination

In [91]:
flag = False
days = 3
filteredData0 = POI_unique.loc[(POI_unique['Destination_name'] == "Los Angeles") & (POI_unique['Trip_type'] == nlp[0])]
filteredData0 = filteredData0.sort_values(by=['POI_reviews_count','Average_Rating'], ascending=False)


if len(nlp) == 2:
    flag = True
    filteredData1 = POI_unique.loc[(POI_unique['Destination_name'] == "Los Angeles") & (POI_unique['Trip_type'] == nlp[1])]
    filteredData1 = filteredData1.sort_values(by=['POI_reviews_count','Average_Rating'], ascending=False)

if flag:
    if days % 2 == 0:
        Trip_type_1 = filteredData0.head(int(days/2))
        Trip_type_2 = filteredData1.head(int(days/2))
    else:
        if len(filteredData0["POI_name"]) > len(filteredData1["POI_name"]):
            Trip_type_1 = filteredData0.head(int((days+1)/2))
            Trip_type_2 = filteredData1.head(days//2)
        else:
            Trip_type_1 = filteredData0.head(days//2)
            Trip_type_2 = filteredData1.head(int((days+1)/2))
    POI_final = pd.concat([Trip_type_1, Trip_type_2])
else:
    POI_final = filteredData0.head(days)
    
POI_final       

,POI_name,Average_Rating,Destination_name,POI_reviews_count,POI_category,POI_description,POI_rating,POI_address,POI_Suggested_Duration,POI_open_close_hours,Review_Sentiment,Trip_type
140,Griffith Observatory,4.5,Los Angeles,20272,Observatories & Planetariums,A public observatory owned and operated by the...,4.5,"2800 E. Observatory Rd., Los Angeles, CA 90027...",Suggested Duration:< 1 hour,Hours\nTue - Fri\n12:00 PM - 10:00 PM\nSat - S...,Positive,Historic
153,Hollywood Walk of Fame,3.5,Los Angeles,17567,"Points of Interest & Landmarks, Historic Walki...",A boulevard where the stars of Hollywood are i...,3.5,"Hollywood Blvd at Vine St, Los Angeles, CA 90028",Suggested Duration:< 1 hour,NaN,Positive,Historic
457,Venice Beach,4.0,Los Angeles,10759,Beaches,"Lively, popular and crowded beach close to Los...",4.0,"Los Angeles, CA 90291",Suggested Duration:2-3 hours,NaN,Positive,Relaxed


## Extracting nearby POIs

In [103]:
nearby = pd.read_csv("final_nearby.csv")

,POI_name,POI_nearby_name,Address,Distance_miles,Distance_minutes
0,Disney California Adventure Park,Magic and Wine Theater With David Minkin,900 S Disneyland Dr Anaheim Majestic Garden Ho...,2.000000,3.0
1,Disney California Adventure Park,Fireworks at Disneyland Park,"1313 Disneyland Drive, Anaheim, CA",3.333333,5.0
2,Disney California Adventure Park,Disneyland Park,"1313 Disneyland Dr, Anaheim, CA 92802",5.333333,8.0
3,Disney California Adventure Park,Star Wars: Galaxy’s Edge,"1313 Disneyland Dr., Anaheim, CA 92802",3.333333,5.0
4,Disney California Adventure Park,Goofy's Playhouse,"1313 Disneyland Drive Mickey's Toontown, Anahe...",3.333333,5.0
...,...,...,...,...,...
3857,Venice Beach,Venice Skatepark,"1500 Ocean Front Walk, Los Angeles, CA 90291-3606",2.000000,3.0
3858,Venice Beach,Muscle Beach Venice,"1800 Ocean Front Walk, Los Angeles, CA 90401-3124",0.666667,1.0
3859,Venice Beach,High Rooftop Lounge,"1697 Pacific Ave, Los Angeles, CA 90291-3613",1.300000,2.0
3860,Venice Beach,Touch of Venice mural,"21 Winward Avenue, Venice, Los Angeles, CA",1.300000,2.0


In [104]:
pois = list(POI_final["POI_name"])

In [105]:
pois

['Griffith Observatory', 'Hollywood Walk of Fame', 'Venice Beach']

In [106]:
fn0 = nearby.loc[(nearby['POI_name'] == pois[0])]
fn0 = fn0.sort_values(by=['Distance_miles'], ascending=True)

In [107]:
fn0.head(3)

,POI_name,POI_nearby_name,Address,Distance_miles,Distance_minutes
654,Griffith Observatory,Charlie Turner's Trail,"2800 E Observatory Ave, Los Angeles, CA 90027-...",0.666667,1.00
655,Griffith Observatory,Solar System Lawn Model,"2800 E Observatory Ave Griffith Observatory, L...",0.666667,1.00
669,Griffith Observatory,Hollyhock House,"4800 Hollywood Blvd Barnsdall Art Park, Los An...",1.300000,1.95


In [108]:
fn1 = nearby.loc[(nearby['POI_name'] == pois[1])]
fn1 = fn1.sort_values(by=['Distance_miles'], ascending=True)
fn1.head(3)

,POI_name,POI_nearby_name,Address,Distance_miles,Distance_minutes
3853,Hollywood Walk of Fame,Pantages Theater,"6233 Hollywood Blvd, Los Angeles, CA 90028-5310",0.666667,1.0
3855,Hollywood Walk of Fame,Iguana Vintage Clothing,"6320 Hollywood Blvd Hollywood, Los Angeles, CA...",0.666667,1.0
3856,Hollywood Walk of Fame,Frolic Room,"6245 Hollywood Blvd, Los Angeles, CA 90028-5310",0.666667,1.0


In [109]:
fn2 = nearby.loc[(nearby['POI_name'] == pois[2])]
fn2 = fn2.sort_values(by=['Distance_miles'], ascending=True)
fn2.head(3)

,POI_name,POI_nearby_name,Address,Distance_miles,Distance_minutes
3858,Venice Beach,Muscle Beach Venice,"1800 Ocean Front Walk, Los Angeles, CA 90401-3124",0.666667,1.0
3861,Venice Beach,Venice art walls,"1800 Ocean Front Walk, Venice, Los Angeles, CA",0.666667,1.0
3859,Venice Beach,High Rooftop Lounge,"1697 Pacific Ave, Los Angeles, CA 90291-3613",1.300000,2.0


In [110]:
POI_final["POI_name"]

140      Griffith Observatory
153    Hollywood Walk of Fame
457              Venice Beach
Name: POI_name, dtype: object